In [ ]:
from google.colab import drive 
drive.mount('/mntDrive')

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [ ]:
! rm -r ocrpostcorrection

In [ ]:
!git clone https://github.com/jvdzwaan/ocrpostcorrection.git

Cloning into 'ocrpostcorrection'...
remote: Enumerating objects: 713, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 713 (delta 81), reused 77 (delta 44), pack-reused 595
Receiving objects: 100% (713/713), 1.16 MiB | 15.60 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [ ]:
!pip install ./ocrpostcorrection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./ocrpostcorrection
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ocrpostcorrection: filename=ocrpostcorrection-0.0.1-py3-none-any.whl size=24546 sha256=210841bf82866a3aa9c9839bb28e75e91dd1ac8200149059a5c92a199f047992
  Stored in directory: /tmp/pip-ephem-wheel-cache-bnjgq3xx/wheels/ce/8c/19/e683c70df08a7eb9cb93fa17f510cfeef9e1e912a83c835a2c
Successfully built ocrpostcorrection
  Attempting uninstall: ocrpostcorrection
    Found existing installation: ocrpostcorrection 0.0.1
    Uninstalling ocrpostcorrection-0.0.1

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading data

In [ ]:
data_base_dir = Path('/Users/janneke/Documents/Documents – Janneke’s MacBook/data/ocrpostcorrection')

In [ ]:
data_base_dir = Path('/mntDrive/MyDrive/data')

In [ ]:
in_file = data_base_dir/'icdar-task2-dataset-20221031'/'task2dataset-no-duplicates.csv'
data = pd.read_csv(in_file, index_col=0)
data = data.fillna('')

In [ ]:
train = data.query('dataset == "train"')
test = data.query('dataset == "test"')

In [ ]:
from ocrpostcorrection.error_correction import generate_vocabs, get_text_transform

vocab_transform = generate_vocabs(train)
text_transform = get_text_transform(vocab_transform)

In [ ]:
from torch.utils.data import DataLoader

from ocrpostcorrection.error_correction import SimpleCorrectionDataset, collate_fn

max_len = 11
batch_size = 128

test_dataset = SimpleCorrectionDataset(test, max_len=max_len)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn(text_transform))

## Load model

In [ ]:
from ocrpostcorrection.error_correction import SimpleCorrectionSeq2seq

hidden_size = 256
dropout = 0.1
model = SimpleCorrectionSeq2seq(len(vocab_transform['ocr']), 
                                hidden_size, 
                                len(vocab_transform['gs']), 
                                dropout, 
                                max_len, 
                                teacher_forcing_ratio=0.5,
                                device=device)
model.to(device)    
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
!ls /mntDrive/MyDrive/data

In [ ]:
model_save_path = data_base_dir/'ocrpostcorrection'/'results'/'simple_correction_model'/'model.rar'

checkpoint = torch.load(model_save_path, map_location=torch.device(device))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                        
model = model.to(device)

In [ ]:
from ocrpostcorrection.error_correction import validate_model

validate_model(model, test_dataloader, max_len, device)

8.245685641283513

In [ ]:
from ocrpostcorrection.error_correction import predict_and_convert_to_str

predictions = predict_and_convert_to_str(model, test_dataloader, vocab_transform['gs'], max_len, device)

100%|██████████| 1453/1453 [03:20<00:00,  7.26it/s]


In [ ]:
test_results = test.query(f'len_ocr < {max_len}').query(f'len_gs < {max_len}').copy()

test_results['pred'] = predictions

In [ ]:
import edlib 

test_results['ed'] = test_results.apply(lambda row: edlib.align(row.ocr, row.gs)['editDistance'], axis=1)
test_results.ed.describe()

count    185866.000000
mean          2.360851
std           1.454669
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          10.000000
Name: ed, dtype: float64

In [ ]:
from ocrpostcorrection.icdar_data import normalized_ed

test_results['ed_norm'] = test_results.apply(lambda row: normalized_ed(row.ed, row.ocr, row.gs), axis=1)
test_results.ed_norm.describe()

count    185866.000000
mean          0.379646
std           0.227861
min           0.100000
25%           0.200000
50%           0.333333
75%           0.500000
max           1.000000
Name: ed_norm, dtype: float64

In [ ]:
import edlib 

test_results['ed_pred'] = test_results.apply(lambda row: edlib.align(row.pred, row.gs)['editDistance'], axis=1)
test_results.ed_pred.describe()

count    185866.000000
mean          1.785055
std           1.676747
min           0.000000
25%           1.000000
50%           1.000000
75%           3.000000
max          11.000000
Name: ed_pred, dtype: float64

In [ ]:
test_results['ed_norm_pred'] = test_results.apply(lambda row: normalized_ed(row.ed_pred, row.pred, row.gs), axis=1)
test_results.ed_norm_pred.describe()

count    185866.000000
mean          0.296892
std           0.273540
min           0.000000
25%           0.100000
50%           0.250000
75%           0.444444
max           1.000000
Name: ed_norm_pred, dtype: float64

In [ ]:
(test_results.pred == test_results.gs).sum()/test_results.shape[0]

0.2497605802029419

In [ ]:
test_results[test_results.pred == test_results.gs].sample(5)

,ocr,gs,ocr_aligned,gs_aligned,start,len_ocr,language,subset,dataset,len_gs,diff,pred,ed,ed_norm,ed_pred,ed_norm_pred
1795813,Serrv,Herr,Serrv,Herr@,1062,5,DE,DE3,test,4,1,Herr,2,0.400000,0,0.0
1723722,EGtube,Stube,EGtube,@Stube,1185,6,DE,DE3,test,5,1,Stube,2,0.333333,0,0.0
1652991,denfen. |,denken.,denfen. |,denken.@@,1265,9,DE,DE3,test,7,2,denken.,3,0.333333,0,0.0
1506077,folteft,ſolteſt,folteft,ſolteſt,920,7,DE,DE6,test,7,0,ſolteſt,2,0.285714,0,0.0
1591872,ffürzen,ſtürzen,ffürzen,ſtürzen,655,7,DE,DE3,test,7,0,ſtürzen,2,0.285714,0,0.0


In [ ]:
out_file = data_base_dir/'ocrpostcorrection'/'results'/'simple_correction_model'/'predictions.csv'
test_results.to_csv(out_file)